# Introduction
Nom: Potvin
Prenom: Ludovic

Ce travail est divise en 3 etapes cle.
- Le remplacement des donnees manquantes.
- L'ajustement des outliers.
- La jointure de d'autre dataset au dataset customer

Le resultat final est dans un seul pipeline.

> Note: Il y a aussi des etapes de normalisation dans le pipeline, tel que OneHotEncoder et StandardScaler, car ils vont facilite le machine learning pour le devoir 2.

# Q3.1 Reproduction des etapes 2 a 4 d'un projet de ML
Cette question sera repondu a travers la reponse au autres questions, pour l'instant simplement importer les donnees.

In [ ]:
import pandas as pd
import numpy as np

num_attribs = ['age', 'pages', 'News_click']
cat_attribs = ['first_item_prize', 'gender', 'ReBuy', 'country', 'revenue']

customer = pd.read_csv('Customer.csv')

# Q3.2 Nettoyage des donnees du dataset de base
Premiere etapes de nettoyage et de prise de conscience du dataset
# Q3.2.1 Remplacement des donnees manquantes

In [ ]:
customer.info()
print(customer.isna().sum())
print(customer.isnull().sum())

question_marks = (customer == '?').sum()
print('==Question mark==')
print(question_marks)

customer.describe(include='all')
customer.head()

In [ ]:
import matplotlib.pyplot as plt
customer.hist(bins=50, figsize=(10,5))
plt.show()

Il n'y a pas de valeur null ou de NaN dans le dataset, il y a cependant quelque '?' dans la column first_item_prize.
La fonction suivante va permettre de les remplacer pour des NaN.
On pourra ensuite remplacer les NaN pour la mediane dans le pipeline.

In [ ]:
def replace_question_with_nan(X):
    X_copy = X.copy()
    X_copy = X_copy.replace('?', np.nan)
    return X_copy

# 3.2.2 Remplacement des donnees aberrantes

In [ ]:
import matplotlib.pyplot as plt

for col in num_attribs:
    plt.figure(figsize=(5,2))
    customer.boxplot(column=col)
    plt.title(f"Boxplot de {col}")
    plt.ylabel(col)
plt.show()

Avec c'est graphique, il est possible d'observer que les pages et les ages ont tout les deux des outliers. La fonction suivante va permettre de ramener les outliers avec les autres.

In [ ]:
def iqr_amplitude(X):
    X_copy = X.copy()
    for col in X_copy:
        col_data = X_copy[col]

        Q1 = np.percentile(col_data, 25)
        Q3 = np.percentile(col_data, 75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        # Remplacer les valeurs < lower par Q1
        X_copy[col] = np.where(X_copy[col] < lower, Q1, X_copy[col])
        # Remplacer les valeurs > upper par Q3
        X_copy[col] = np.where(X_copy[col] > upper, Q3, X_copy[col])

    return X_copy

# Q3.3 Enrichissement des donnees
## Exploration des autres datasets

In [ ]:
GDP = pd.read_csv('CountryGDP.csv')
GDP.info()
GDP.head()

In [ ]:
population = pd.read_csv('CountryPopulation.csv')
population.info()
population.head()

La fonction suivante va permettre de joindre les datasets de CountryPopulation et CountryGDP au dataset customer.

In [ ]:
def enrichissement_population_pib(X, PIB=True):
    X_copy = X.copy()

    population = pd.read_csv('CountryPopulation.csv')
    population.rename(columns={"Country": "country"}, inplace=True)
    X_copy = pd.merge(X_copy, population, on='country') 
    
    if PIB:
        GDP = pd.read_csv('CountryGDP.csv')
        GDP.rename(columns={"Country": "country"}, inplace=True)
        X_copy = pd.merge(X_copy, GDP, on='country') 

    return X_copy

# Pipeline final
Voici le pipeline avec chaque etapes. A note qu'il est possible d'enlever le StandardScaler et le OnHotEncoder pour lire plus clairement le resultat. Les deux etapes sont presentes pour facilite le machine learning dans le devoir 2.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer

enrichissement_transformer = FunctionTransformer(enrichissement_population_pib, validate=False, kw_args={"PIB": True})

numeric_transformer = Pipeline([
    ("Replace question mark", FunctionTransformer(replace_question_with_nan, validate=False)),
    ("iqr outliers", FunctionTransformer(iqr_amplitude, validate=False)),
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

preprocessor = ColumnTransformer([
    ("num", numeric_transformer, num_attribs + ['population', 'GDP_inhab']),
    ("cat", OneHotEncoder(), cat_attribs),
])

full_pipeline = Pipeline([
    ('enrich', enrichissement_transformer),
    ('preprocessor', preprocessor),
])

clean_customer = full_pipeline.fit_transform(customer)
customer_array = clean_customer.toarray()
pd.DataFrame(customer_array).info()
pd.DataFrame(customer_array).head()